In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_48c_J8_nf8_v1.0.2.pth

In [2]:
!nvidia-smi --query-gpu=name --format=csv,noheader | head -n 1

NVIDIA GeForce RTX 4090


In [1]:
import io
import time
import torch
import numpy as np
import PIL
import torchaudio
import matplotlib.pyplot as plt
from einops import rearrange
from IPython.display import Audio
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc
from spauq.core.metrics import spauq_eval
import cdpam
class Config: pass

In [2]:
checkpoint = torch.load("Stereo_Li_48c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
cdpam_loss = cdpam.CDPAM()

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


In [4]:
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
max_duration = 44100*120
def walloc_compress(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio']['bytes'])
        x = x[:,:,:max_duration]
        L = x.shape[-1]
        x_padded = pad(x.unsqueeze(0), 2**16).to(device)
        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        ℓ = Y.shape[-1]
        Y = pad(Y,256)
        Y = rearrange(Y, 'b c (w h) -> b c w h', h=256).to("cpu")
        webp = walloc.latent_to_pil(Y,codec.latent_bits,3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
    
        t0 = time.time()
        Y = walloc.pil_to_latent([PIL.Image.open(buff)], codec.latent_dim, codec.latent_bits, 3).to(device)
        X_hat = codec.decoder(rearrange(Y.to(device), 'b c h w -> b c (h w)')[:,:,:ℓ])
        x_hat = codec.wavelet_synthesis(X_hat,codec.J)
        x_hat = codec.post(x_hat)
        x_hat = x_hat[0,:,:L].clamp(-1., 1.)
        decode_time = time.time() - t0
    
        bps = 8*len(webp_bytes)/(x.numel())
        PSNR = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(x,x_hat.to("cpu")))
        SDR = spauq_eval(x,x_hat.to("cpu"),fs=fs)
        SSDR = SDR['SSR']
        SRDR = SDR['SRR']
        cdpam = cdpam_loss.forward(x.to(device),x_hat).mean().item()
        
    return {
        'compressed': webp_bytes,
        'encode_time': encode_time,
        'decode_time': decode_time,
        'bps': bps,
        'PSNR': PSNR,
        'SSDR': SSDR,
        'SRDR': SRDR,
        'CDPAM': cdpam
    }

In [7]:
device = "cuda"
codec = codec.to(device)
gpu = MUSDB.map(walloc_compress)
gpu = gpu.cast_column('compressed',Image())

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.18 GiB. GPU 0 has a total capacity of 23.65 GiB of which 8.03 GiB is free. Including non-PyTorch memory, this process has 15.59 GiB memory in use. Of the allocated memory 14.96 GiB is allocated by PyTorch, and 187.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
# def walloc_compress_cpu(sample):
#     with torch.no_grad():
#         img = sample['image']
#         x = PILToTensor()(img).to(torch.float)
#         x = (x/255 - 0.5).unsqueeze(0).to(device)
#         H, W = x.size(2), x.size(3)
#         x_padded = walloc.pad(x,p=8)

#         t0 = time.time()
#         X = codec.wavelet_analysis(x_padded,codec.J)
#         Y = codec.encoder(X)
#         webp = walloc.latent_to_pil(Y.to("cpu"),codec.latent_bits, 3)[0]
#         buff = io.BytesIO()
#         webp.save(buff, format='WEBP', lossless=True)
#         webp_bytes = buff.getbuffer()
#         encode_time = time.time() - t0

#         t0 = time.time()
#         Y = walloc.pil_to_latent([PIL.Image.open(buff)], codec.latent_dim, codec.latent_bits, 3).to(device)
#         X_hat = codec.decoder(Y)
#         x_hat = codec.wavelet_synthesis(X_hat,codec.J)
#         x_hat = codec.clamp(x_hat)
#         decode_time = time.time() - t0
                
#     return {
#         'cpu_encode_time': encode_time,
#         'cpu_decode_time': decode_time,
#     }

In [8]:
# device = "cpu"
# codec = codec.to(device)
# cpu = LSDIR.map(walloc_compress_cpu)
# combined = gpu.add_column('cpu_encode_time',cpu['cpu_encode_time'])
# combined = combined.add_column('cpu_decode_time',cpu['cpu_decode_time'])

In [9]:
# metrics = [
#      'encode_time',
#      'decode_time',
#      'bpp',
#      'PSNR',
#      'MSSIM',
#      'LPIPS_dB',
#      'DISTS_dB', 
#      'cpu_encode_time',
#      'cpu_decode_time',
# ]

In [10]:
# for metric in metrics:
#     μ = np.mean(combined[metric])
#     print(f"{metric}: {μ}")

---

In [11]:
# L = 1048064
# sample = MUSDB[5]
# x, fs = torchaudio.load(sample['audio']['bytes'])
# x = x[:,:L]
# codec = codec.to("cuda")
# x = x.to("cuda").unsqueeze(0)
# with torch.no_grad():
#     X = codec.wavelet_analysis(x,codec.J)
#     Y = codec.encoder(X)
#     Z = codec.decoder(Y)
#     z = codec.wavelet_synthesis(Z,codec.J)
#     z = codec.post(z)
# x = x.to("cpu")[0]
# z = z.to("cpu")[0]

In [12]:
# Audio(x.numpy(),rate=44100)

In [13]:
# Audio(z.numpy(),rate=44100)

In [14]:
# start, end = 56500, 57000
# plt.figure(figsize=(8, 4), dpi=180)
# plt.plot(x[0, start:end], alpha=0.5, c='b', label='Ch.1 (Uncompressed)')
# plt.plot(z[0, start:end], alpha=0.5, c='g', label='Ch.1 (WaLLoC)')
# plt.plot(x[1, start:end], alpha=0.5, c='r', label='Ch.2 (Uncompressed)')
# plt.plot(z[1, start:end], alpha=0.5, c='purple', label='Ch.2 (WaLLoC)')

# plt.ylim([-1,0.6])
# plt.legend(loc='lower center')
# plt.box(False)
# plt.xticks([])
# plt.yticks([])
# # plt.savefig("test.svg")